In [31]:
from dotenv import load_dotenv
import sys

sys.path.append("../../common")
load_dotenv()

True

In [32]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="20.Project")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 20.Project]


In [33]:
email_conversation = """
제목: 블록체인과 AI 기술 융합 프로젝트 논의 요청

보내는 사람: 박은빈 (기획팀)
이메일: silverempty@gmail.com

받는 사람: 김태리 (개발팀)
이메일: kimterry@gmail.com

안녕하세요, 김태리 개발팀장님,
기획팀 박은빈입니다.

최근 기획팀에서 논의 중인 블록체인과 AI 기술 융합 프로젝트와 관련하여, 구체적인 방향을 설정하기 위해 개발팀의 전문적인 의견을 듣고자 합니다.
이에 따라 아래와 같이 회의 일정을 제안드리오니, 확인 후 참석 부탁드립니다.

회의 일정
일시: 2024년 11월 28일(목) 오후 2시
장소: 본사 3층 회의실 A
참석 대상: 기획팀, 개발팀

주요 논의 내용
1. 블록체인 데이터 활용 방안
- 스마트 계약 데이터를 AI 학습 데이터로 변환하는 과정에서의 기술적 고려사항
- 트랜잭션 데이터의 실시간 분석 가능성

2. AI 모델의 역할
- 블록체인 네트워크 내 이상 감지 모델 설계
- 사용자 행동 데이터를 분석해 트렌드 예측 시스템 구축

3. 기술 스택 제안
- 블록체인 네트워크와 호환 가능한 AI 모델 및 프레임워크 논의
- 시스템 간 데이터 통합 및 처리 효율성 확보 방안

이번 회의를 통해 프로젝트의 기술적 타당성을 검토하고, 초기 기술 스펙 초안을 확정하는 것이 목표입니다.
혹시 일정에 어려움이 있으시다면, 조율이 가능하니 미리 말씀 부탁드립니다.

바쁘신 와중에도 시간 내주셔서 감사합니다.
회의에서 뵙겠습니다!

감사합니다.
박은빈 드림

박은빈
기획팀
E: silverempty@gmail.com
T: 010-1234-5678
"""

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class EmailSummury(BaseModel):  # BaseModel 기능을 상속받음
    sender_name: str = Field(description="발신자 이름")
    sender_department: str = Field(description="발신자 부서")
    sender_email: str = Field(description="발신자 이메일")
    subject: str = Field(description="메일 제목")
    summury: str = Field(description="메일 본문의 중요 내용 요약")
    keyword: str = Field(description="메일 본문의 핵심 키워드")
    date: str = Field(description="회의 일정 일시")

In [44]:
## LCEL 구조
output_parser = PydanticOutputParser(pydantic_object=EmailSummury)

template = """
    당신은 AI 어시드던트입니다. 아래 내용에 대한 질문을 한글로 대답해주세요.
    
    #Qustion:
    다음의 이메일 내용 중에서 주요 내용을 추출해주세요.
    
    #EMAIL CONVERSATION:
    {email_conversation}
    
    #FORMAT:
    {format_instruction}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instruction": output_parser.get_format_instructions()},
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

In [45]:
response = chain.invoke({"email_conversation": email_conversation})
response

EmailSummury(sender_name='박은빈', sender_department='기획팀', sender_email='silverempty@gmail.com', subject='블록체인과 AI 기술 융합 프로젝트 논의 요청', summury='블록체인과 AI 기술 융합 프로젝트에 대한 개발팀의 의견을 듣기 위한 회의 요청.', keyword='블록체인, AI, 융합 프로젝트, 회의', date='2024년 11월 28일(목) 오후 2시')

### 검색 SERP API

홈페이지 : https://serpapi.com/integrations/python

In [41]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "2"}

search = SerpAPIWrapper(params=params)

In [49]:
search_query = f"{response.keyword}"

search_result = search.run(search_query)

search_result = eval(search_result)

search_result

['14일 서울 신라호텔에서 진행된 이번 행사는 블록체인 기술의 발전과 융합 가능성에 대해 다루며 전 세계 금융 및 정보 시스템의 변화를 주제로 한 케이시 ...',
 'AI와 블록체인의 융합에 대해서 회의적인 시각을 내는 것은 비단 부테린 뿐만이 아닙니다. 알고리즘의 투명성과 보안의 위협에 대해서도 끊임없는 ...']

구조화된 답변을 다음 체인의 입력으로 추가합니다.

In [58]:
from langchain_core.prompts import PromptTemplate

template = """
당신은 미에일의 주요 정보를 바탕으로 요약 정리해주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(Format)에 맞추어 요약을 작성해주세요.
답변에는 카테고리 별로 emoji를 적극 사용해서 답변해 주세요.

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Department: {department} 
- Email: {email}
- Subject: {subject}
- Summury: {summury}
- Date: {date} 

#Format(in markdown format):
🙇‍♂️ 보낸 사람:
- (보낸 사람 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

ℹ️ 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 정보)

❗️ 주요 내용:
- (이메일 제목, 요약 or 주요 내용)

🗓️ 일정
- (회의 날짜 및 시간)

#Answer
"""

report_prompt = PromptTemplate.from_template(template=template)

In [59]:
from langchain_core.output_parsers import StrOutputParser

report_chain = report_prompt | llm | StrOutputParser()

report_response = report_chain.invoke(
    {
        "sender": response.sender_name,
        "additional_information": search_result,
        "department": response.sender_department,
        "email": response.sender_email,
        "subject": response.subject,
        "summury": response.summury,
        "date": response.date,
    }
)

print(report_response)

🙇‍♂️ 보낸 사람:
- 박은빈, 기획팀

📧 이메일 주소:
- silverempty@gmail.com

ℹ️ 보낸 사람과 관련하여 검색된 추가 정보:
- 14일 서울 신라호텔에서 진행된 이번 행사는 블록체인 기술의 발전과 융합 가능성에 대해 다루며 전 세계 금융 및 정보 시스템의 변화를 주제로 한 케이시 ... 
- AI와 블록체인의 융합에 대해서 회의적인 시각을 내는 것은 비단 부테린 뿐만이 아닙니다. 알고리즘의 투명성과 보안의 위협에 대해서도 끊임없는 ...

❗️ 주요 내용:
- 블록체인과 AI 기술 융합 프로젝트 논의 요청, 개발팀의 의견을 듣기 위한 회의 요청.

🗓️ 일정
- 2024년 11월 28일(목) 오후 2시
